In [9]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from PIL import ImageEnhance

import numpy as np

In [14]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize(224),  # Resize to 224x224 to match AlexNet input size
    transforms.ToTensor(),   # Convert the image to a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])

class ImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        pass

In [4]:
droplet_image = 'data/path'

# Load the image
img = Image.open(droplet_image)

In [10]:
# img which is a PIL image to numpy array
img_array = np.array(img)

In [12]:
img_array.shape

(50, 50, 3)